In [40]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms
import numpy as np

ModuleNotFoundError: No module named 'torchvision'

# Data Loading

In [33]:
PATH = "/Users/mikolajstarczewski/Desktop/Magisterka/NN_with_py/nn_with_py/data_files/"

X_train = np.load(PATH + "X_train.npy")
X_test = np.load(PATH + "X_test.npy")
Y_train = np.load(PATH + "Y_train_ohe.npy")
Y_test = np.load(PATH + "Y_test_ohe.npy")

X_train = X_train.reshape(X_train.shape[0], -1)
X_test = X_test.reshape(X_test.shape[0], -1)

# Define the neural network architecture

In [34]:
class NeuralNet(nn.Module):
    def __init__(self):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(1024, 1024)  # Adjust input size accordingly if different
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(1024, 512)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(512, 256)
        self.relu3 = nn.ReLU()
        self.dropout1 = nn.Dropout(0.2)
        self.fc4 = nn.Linear(256, 128)
        self.relu4 = nn.ReLU()
        self.dropout2 = nn.Dropout(0.2)
        self.fc5 = nn.Linear(128, 89)
        self.softmax = nn.Softmax(dim=1)
    
    def forward(self, x):
        x = self.fc1(x)
        x = self.relu1(x)
        x = self.fc2(x)
        x = self.relu2(x)
        x = self.fc3(x)
        x = self.relu3(x)
        x = self.dropout1(x)
        x = self.fc4(x)
        x = self.relu4(x)
        x = self.dropout2(x)
        x = self.fc5(x)
        x = self.softmax(x)
        return x

In [35]:
# Initialize the model
model = NeuralNet()

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=5e-6)

In [36]:
# Training loop
def train_model(model, X_train, Y_train, X_test, Y_test, epochs=30, batch_size=128):
    train_loader = torch.utils.data.DataLoader(
        torch.utils.data.TensorDataset(X_train, Y_train), batch_size=batch_size, shuffle=True)
    
    test_loader = torch.utils.data.DataLoader(
        torch.utils.data.TensorDataset(X_test, Y_test), batch_size=batch_size)
    
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        for i, (inputs, labels) in enumerate(train_loader):
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
            
            if i % 1000 == 999:  # Print every 1000 batches
                print(f'Epoch [{epoch + 1}/{epochs}], Step [{i + 1}/{len(train_loader)}], Loss: {loss.item():.4f}')
        
        # Validation step
        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for inputs, labels in test_loader:
                outputs = model(inputs)
                _, predicted = torch.max(outputs, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        
        accuracy = 100 * correct / total
        print(f'Epoch [{epoch + 1}/{epochs}], Validation Accuracy: {accuracy:.2f}%')


In [37]:
X_train = torch.tensor(X_train, dtype=torch.int)
Y_train = torch.tensor(Y_train.argmax(axis=1), dtype=torch.int)  # Convert one-hot to class index
X_test = torch.tensor(X_test, dtype=torch.int)
Y_test = torch.tensor(Y_test.argmax(axis=1), dtype=torch.int)

In [38]:
# Train the model
train_model(model, X_train, Y_train, X_test, Y_test)

Epoch [1/30], Step [1000/3489], Loss: 4.4249
Epoch [1/30], Step [2000/3489], Loss: 4.3738
Epoch [1/30], Step [3000/3489], Loss: 4.3161
Epoch [1/30], Validation Accuracy: 17.10%
Epoch [2/30], Step [1000/3489], Loss: 4.3121
Epoch [2/30], Step [2000/3489], Loss: 4.3433
Epoch [2/30], Step [3000/3489], Loss: 4.2891
Epoch [2/30], Validation Accuracy: 19.84%
Epoch [3/30], Step [1000/3489], Loss: 4.2984
Epoch [3/30], Step [2000/3489], Loss: 4.2575
Epoch [3/30], Step [3000/3489], Loss: 4.2654
Epoch [3/30], Validation Accuracy: 20.20%
Epoch [4/30], Step [1000/3489], Loss: 4.3587
Epoch [4/30], Step [2000/3489], Loss: 4.2657


KeyboardInterrupt: 

# CNN

<h3> Define the CNN model


RuntimeError: shape '[446524, 1, 28, 28]' is invalid for input of size 457240576